In [1]:
import os
from joblib import load
from catboost import CatBoostClassifier

# Import models

In [2]:
rf_models = []
path = 'models/rf_0.500/'
for filename in os.listdir(path):
    model = load(path+filename)
    rf_models.append(model)
    
cb_models = []
path = 'models/cb_0.584/'
for filename in os.listdir(path):
    model = CatBoostClassifier().load_model(path+filename)
    cb_models.append(model)

In [3]:
all_models = rf_models+cb_models

# Test set

In [4]:
import pandas as pd
import numpy as np
from utils.eda import reduce_mem_usage, get_cats_nums

In [5]:
path = "../data/mlhackfest-haribon1//"
test = pd.read_csv(path+"test.csv")
test.columns = [f.lower() for f in test.columns]
index = 'indexid'
label = 'scientific name'
cats, nums = get_cats_nums(test)
cats = [x for x in cats if x not in [index, label]]
nums = [x for x in nums if x not in [index, label]]
feats = [f for f in cats if f not in [index, label]] + [f for f in nums if f not in [index, label]]
test = reduce_mem_usage(test, verbose=1)

Memory usage of properties dataframe is : 0.0123291015625  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  0.0049285888671875  MB
This is  39.975247524752476 % of the initial size


In [6]:
all_preds = []
for model in all_models:
    model_preds = model.predict_proba(test[feats])
    all_preds.append(model_preds)
    print(model)

all_preds = np.array(all_preds).mean(axis=0).argmax(axis=1)
test['Predicted'] = all_preds

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=13, verbose=0,
                       warm_start=False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=13, verbose=0,
                       warm_start=False)
RandomForestClassifier(bootstrap

In [7]:
mapping = {0: 'Alcedo argentata',
           1: 'Anas luzonica',
           2: 'Cacatua haematuropygia',
           3: 'Egretta eulophotes',
           4: 'Pithecophaga jeffeyri'}

test['Predicted'] = test['Predicted'].map(mapping)

In [8]:
test.rename(columns={
    'indexid': 'indexId'
}, inplace=True)

In [9]:
sub_path = "submission/haribon_submission/"
filename = "together1"
test[['indexId', 'Predicted']].to_csv(sub_path+filename+".csv", index=False)

In [10]:
test['Predicted'].value_counts()

Pithecophaga jeffeyri     34
Cacatua haematuropygia    30
Egretta eulophotes         7
Anas luzonica              5
Alcedo argentata           4
Name: Predicted, dtype: int64